In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

### HANDLING MISSING DATA ###

# Read training data
train_data_path = 'train.csv'
home_train_data = pd.read_csv(train_data_path, index_col='Id')

# Read testing data
test_data_path = 'test.csv'
home_test_data = pd.read_csv(test_data_path, index_col='Id')


# Remove rows in the training data with missing values 
# NOTE: These become 'NoneType' object with no length
home_train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = home_train_data.SalePrice
home_train_data.drop(['SalePrice'], axis=1, inplace=True)

print("Length of y:", len(y))

Length of y: 1460


In [2]:
# To keep thing simple, use only numerical predictors (training data)
X = home_train_data.select_dtypes(exclude=['object'])

# Top keep things simple, use only numerical predictors (testing data)
X_test = home_test_data.select_dtypes(exclude=['object'])

# NOTE: Again, Training & Testing data have different lengths

print("X_num:", len(X))
print("X_num_test:", len(X_test))

X_num: 1460
X_num_test: 1459


In [3]:
# Break off validation set from training data

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [4]:
# This should have 36 columns
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


In [5]:
### STEP 1: Preliminary Investigation

# Shape of training data (num_rows, num_columns)

print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [6]:
# Use score_dataset() function to compare different approaches to dealing with missing values

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    # specify model
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    # fit model with training data
    model.fit(X_train, y_train)
    # use model to predict on validation set
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [8]:
### STEP 2: Drop Columns with Missing Values

# Get name of columns with missing values: LotFrontage, MasVnrArea, GarageYrBlt
missing_cols = ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

# Drop columens (w/ missing values) from both training and validation data
# Should have 36 - 3 = 33 columns
reduced_X_train = X_train.drop(missing_cols, axis=1)
reduced_X_valid = X_valid.drop(missing_cols, axis=1)

reduced_X_train.head()
reduced_X_valid.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
530,20,32668,6,3,1957,1975,1219,0,816,2035,...,484,0,0,200,0,0,0,0,3,2007
492,50,9490,6,7,1941,1950,403,165,238,806,...,240,0,0,32,0,0,0,0,8,2006
460,50,7015,5,4,1950,1950,185,0,524,709,...,352,0,0,248,0,0,0,0,7,2009
280,60,10005,7,5,1977,1977,392,0,768,1160,...,505,288,117,0,0,0,0,0,3,2008
656,160,1680,6,5,1971,1971,0,0,525,525,...,264,0,0,0,0,0,0,0,3,2010


In [9]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17837.82570776256


In [10]:
### STEP 3: Alternative Way of Handling Missing Data (IMPUTATION)

# note: fit_transform is fitting the model, then transforming (imputing)
# note: transform is just transforming

from sklearn.impute import SimpleImputer

# Fill in the lines below: imputation
my_imputer = SimpleImputer() 
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18062.894611872147


In [11]:
### STEP 4: Generate Test Predictions

# Preprocessed training and validation features
final_imputer = SimpleImputer(strategy="median")
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))

# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your approach):
17791.59899543379


In [12]:
# Preprocess test data
final_X_test = pd.DataFrame(final_imputer.transform(X_test))

# Get test predictions
preds_test = model.predict(final_X_test)

# print output
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})

print(output)

# OPTIONAL: Save test predictions to file
# output.to_csv('submission3.csv', index=False)

        Id  SalePrice
0     1461  125985.50
1     1462  154599.50
2     1463  180070.24
3     1464  183544.50
4     1465  197549.92
...    ...        ...
1454  2915   87437.12
1455  2916   86994.50
1456  2917  154751.44
1457  2918  107387.00
1458  2919  229281.55

[1459 rows x 2 columns]
